In [12]:
# Day 1
import numpy as np

modules = [
    148216,142030,129401,74642,108051,54128,145495,67818,120225,67113,107672,101032,147714,55788,87732,73681,114646,76586,116436,139788,125150,136675,90527,74674,105505,146059,52735,101389,108121,62897,132337,51963,129188,122308,84677,66433,118374,66822,94714,101162,54030,136580,55677,114051,133898,95026,112964,68662,85139,53559,84703,92053,132197,60130,63184,86182,113038,52659,140463,123234,97887,70216,131832,108162,116759,111828,132815,113476,127734,134545,99643,141911,74705,65720,95640,51581,66787,147590,72937,148774,119881,139875,131976,68238,100342,134691,112320,86107,100045,120458,54459,52047,108226,102138,141233,54452,67859,105132,81903,104282,
]

def fuel_by_mass(mass):
    return (mass // 3) - 2


@np.vectorize
def calculate_module_fuel(module_weight):
    total = fuel_by_mass(module_weight)
    fuel_mass = fuel_by_mass(total)

    while fuel_mass > 0:
        total += fuel_mass
        fuel_mass = fuel_by_mass(fuel_mass)

    return total

required_fuel = 0

for module_weight in modules:
    required_fuel += calculate_fuel(module_weight)

sum(calculate_module_fuel(modules))

In [64]:
# Day 2
import operator

def run_intcode(intcode):
    opcodes = {1: operator.add, 2: operator.mul}

    for i in range(0, len(intcode), 4):
        if intcode[i] == 99:
            return intcode
        elif intcode[i] in opcodes:
            opcode, L1, L2, out = intcode[i:i+4]
            intcode[out] = opcodes.get(opcode)(intcode[L1], intcode[L2])
        else:
            raise ValueError("Error")
    

L = [1,45,59,3,1,1,2,3,1,3,4,3,1,5,0,3,2,1,10,19,1,6,19,23,2,23,6,27,2,6,27,31,2,13,31,35,1,10,35,39,2,39,13,43,1,43,13,47,1,6,47,51,1,10,51,55,2,55,6,59,1,5,59,63,2,9,63,67,1,6,67,71,2,9,71,75,1,6,75,79,2,79,13,83,1,83,10,87,1,13,87,91,1,91,10,95,2,9,95,99,1,5,99,103,2,10,103,107,1,107,2,111,1,111,5,0,99,2,14,0,0]
run_intcode(L)

In [96]:
# Day 3
ip1 = "R1009,U34,L600,U800,R387,D247,R76,U797,R79,D582,L325,D236,R287,U799,R760,U2,L261,D965,R854,D901,R527,D998,R247,U835,L29,U525,L10,D351,L599,D653,L39,D112,R579,D650,L539,D974,R290,U729,L117,D112,L926,U270,L158,D800,L291,U710,L28,D211,R700,U691,L488,D307,R448,U527,L9,D950,L535,D281,L683,U576,L372,U849,R485,D237,L691,U453,L667,U856,R832,U956,L47,D951,R171,U484,R651,D731,L768,D44,R292,U107,L237,U731,L795,D460,R781,U77,L316,U873,L994,D322,L479,U121,R754,U68,L454,D162,L308,D986,L893,D808,R929,D328,L591,D718,R616,U139,R221,U124,R477,U614,L439,D329,R217,D157,L65,D460,R523,U955,R512,D458,L823,D975,R506,D870,R176,U558,R935,U319,L281,D470,L285,U639,L974,U186,L874,U487,L979,D95,R988,U398,R776,D637,R75,U331,R746,D603,R102,U978,R702,U89,L48,D757,L173,D422,L394,U800,R955,U644,R911,D327,R471,D313,L982,D93,R998,U549,R210,D640,R332,U566,R736,U302,L69,U677,L137,U674,R204,D720,R868,U143,L635,D177,L277,D749,R180,D432,R451,D426,R559,U964,L35,U452,L848,D707,R758,D41,R889,D966,R460,U11,R819,D30,L953,U150,L621,U915,R400,D723,R299,D93,L987,D790,L541,U864,R711,D968,L2,D963,L996,D260,L824,D765,L617,U257,R175,U786,L873,D118,L433,U246,R821,D308,L468,U53,R859,U806,L197,D663,R540,D84,L398,D945,L999,U114,L731,D676,L538,U680,R519,U313,R699,U746,R471,D393,L902,U697,R542,D385,R183,U463,R276,U990,R111,U709,R726,D996,L728,D215,R726,D911,L199,D484,R282,U129,L329,U309,L270,U990,L813,U242,L353,D741,R447,D253,L556,U487,L102,D747,L965,D743,R768,U589,R657,D910,L760,D981,L982,D292,R730,U236,L831"
ip2 = "L1000,U720,R111,D390,L400,U931,R961,D366,L172,D434,R514,D185,L555,D91,R644,U693,L902,U833,L28,U136,R204,D897,L18,D601,R855,U409,R567,U57,L561,D598,R399,D238,R37,U478,R792,D587,R740,D647,L593,U576,L662,U389,R540,U359,R547,D449,R518,D747,L887,U421,R153,D365,L357,U495,L374,D27,L338,D57,R431,U796,L487,D480,L273,U662,R874,D29,R596,D166,R167,D788,R175,D395,L739,U180,R145,U824,L156,D387,R427,U167,R268,D653,L371,D467,L216,U23,L930,D494,L76,U338,R813,U373,R237,D1,R706,U37,R202,D187,L905,D431,R787,D391,R576,D370,R320,U225,L901,D921,R656,U517,R782,D965,L849,U241,L160,U792,L587,U408,L750,U21,R317,U919,L449,D691,L895,D853,L547,D178,R793,D921,L873,D962,L232,U690,L815,U309,R455,U156,L200,U34,L761,U402,R278,U952,L294,D183,R475,U770,L375,D117,R58,D905,L580,U240,R263,U549,R771,U512,L20,D996,L265,U619,L742,U754,L68,D824,R694,D678,R412,D321,R611,U325,L874,U776,L907,U39,R568,D485,R528,D197,R487,D920,R879,D935,R107,U897,L263,D979,L420,U498,L757,D348,L279,U266,R699,D729,R65,U672,L945,U780,L339,U324,R927,U357,R324,U435,R602,D245,L456,D161,L537,U740,R454,U211,L952,D356,L317,U456,L6,D718,L389,D554,L366,D141,R543,U756,R334,U209,L207,U726,R375,U59,L238,D118,L514,D390,R212,U272,L350,U898,L105,U514,L591,U839,L767,U651,R298,U726,L429,U350,L53,U789,R9,D295,L558,U9,L515,D177,L430,U158,L959,U601,L994,U635,L252,D159,R155,U601,L809,D5,R47,U567,R328,U559,R149,U43,L612,U428,R694,D568,L80,U80,R983,D143,R612,U735,L10,D697,L640,D788,R714,U555,L139,U396,L830,D825,R928,D25,L889,U973,L343"


def build_wire(instructions):
    def parse_instruction(instruction):
        direction, *steps = instruction
        return direction, int("".join(steps))

    point = [0, 0]
    wire = set()
    steps = 0
    min_steps = {}

    for instruction in instructions:
        direction, units = parse_instruction(instruction)

        if direction in {"R", "U"}:
            sign = 1
        else:
            sign = -1

        if direction in {"L", "R"}:
            axis = 0
        else:
            axis = 1

        for _ in range(units):
            point[axis] += sign
            new_point = tuple(point)
            wire.add(new_point)

            steps += 1

            if new_point not in min_steps:
                min_steps[new_point] = steps

    return {"coordinates": wire, "steps": min_steps}


inputs = [x.split(",") for x in (ip1, ip2)]

wires = [build_wire(ip) for ip in inputs]

crosses = wires[0]["coordinates"].intersection(wires[1]["coordinates"])

cross_steps = (wires[0]["steps"][x] + wires[1]["steps"][x] for x in crosses)

print(min(cross_steps))

In [296]:
# Day 4
"""
Part 1

--- Day 4: Secure Container ---
You arrive at the Venus fuel depot only to discover it's protected by a password. The Elves had written the password on a sticky note, but someone threw it out.

However, they do remember a few key facts about the password:

It is a six-digit number.
The value is within the range given in your puzzle input.
Two adjacent digits are the same (like 22 in 122345).
Going from left to right, the digits never decrease; they only ever increase or stay the same (like 111123 or 135679).
Other than the range rule, the following are true:

111111 meets these criteria (double 11, never decreases).
223450 does not meet these criteria (decreasing pair of digits 50).
123789 does not meet these criteria (no double).
How many different passwords within the range given in your puzzle input meet these criteria?

Your puzzle input is 171309-643603.

"""
def possibilities(low, high):
    def has_doubled(n):
        n = str(n)
        for i, x in enumerate(n):
            if i == len(n)-1:
                break
            elif x == n[i+1]:
                return True
     
    def is_asc(n):
        n = str(n)
        for i, x in enumerate(n):
            if x > n[i+1]:
                return False
            if i == len(n)-2:
                break
        
        return True

    return [x for x in range(low, high) if has_doubled(x) and is_asc(x)]
        
    
"""
--- Part Two ---
An Elf just remembered one more important detail: the two adjacent matching digits are not part of a larger group of matching digits.

Given this additional criterion, but still ignoring the range rule, the following are now true:

112233 meets these criteria because the digits never decrease and all repeated digits are exactly two digits long.
123444 no longer meets the criteria (the repeated 44 is part of a larger group of 444).
111122 meets the criteria (even though 1 is repeated more than twice, it still contains a double 22).
How many different passwords within the range given in your puzzle input meet all of the criteria?

Your puzzle input is still 171309-643603."""

def possibilities_p2(low, high):
    def has_doubled_strict(n):
        n = str(n)
        for i, x in enumerate(n):
            if i == len(n)-1:
                break
            elif x == n[i+1] and ((i == len(n)-2) or x != n[i+2]):
                if x == n[i-1]:
                    continue
                else:
                    return True
        return False
     
    def is_asc(n):
        n = str(n)
        for i, x in enumerate(n):
            if x > n[i+1]:
                return False
            if i == len(n)-2:
                break
        
        return True

    return [x for x in range(low, high) if has_doubled_strict(x) and is_asc(x)]

low = 171309
high = 643603
print(f"""
P1: {len(set(possibilities(low,high)))}
P2: {len(set(possibilities_p2(low,high)))}""")


P1: 1625
P2: 1111


In [294]:
low = 171309
high = 643603

len(set(possibilities_p2(low,high)))

1111

In [291]:
def has_doubled_strict(n):
    n = str(n)
    for i, x in enumerate(n):
        if i == len(n)-1:
            break
        elif x == n[i+1] and ((i == len(n)-2) or x != n[i+2]):
            if x == n[i-1]:
                continue
            return True
    return False

In [292]:
has_doubled_strict(1111444666)

False

In [234]:
for x, y in enumerate(range(2)):
    print(x,y)

0 0
1 1
